In [ ]:
import serial
import time
import pygame

# Replace these with the actual serial port names for your ESP32 boards
ESP32_1_PORT = 'com2'
ESP32_2_PORT = 'com1'

# Initialize Pygame
pygame.init()

# Set the screen dimensions (you can adjust them based on your preference)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 400
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))


def map_values(value, in_min, in_max, out_min, out_max):
    # Map the input value from one range to another
    return (value - in_min) * (out_max - out_min) / (in_max - in_min) + out_min


def send_command_to_esp32(esp32_port, x, y):
    # Open the serial connection
    ser = serial.Serial(esp32_port, 9600, timeout=1)

    # Convert x and y (0-255) to voltage (0-3V)
    voltage_x = map_values(x, 0, 255, 0, 3)
    voltage_y = map_values(y, 0, 255, 0, 3)

    # Send the voltages as strings to the ESP32
    ser.write(f"{voltage_x:.2f},{voltage_y:.2f}\n".encode())

    # Close the serial connection
    ser.close()


if __name__ == "_main_":
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the state of all keyboard keys
        keys = pygame.key.get_pressed()

        # Adjust the speed based on your preference
        speed = 5

        # Check the arrow keys and update x_coord and y_coord accordingly
        x_coord = 128  # Initial value at the center
        y_coord = 128  # Initial value at the center

        if keys[pygame.K_LEFT]:
            x_coord -= speed
        if keys[pygame.K_RIGHT]:
            x_coord += speed
        if keys[pygame.K_UP]:
            y_coord -= speed
        if keys[pygame.K_DOWN]:
            y_coord += speed

        # Limit x_coord and y_coord to the range 0-255
        x_coord = max(0, min(255, x_coord))
        y_coord = max(0, min(255, y_coord))

        # Send commands to the respective ESP32s
        send_command_to_esp32(ESP32_1_PORT, x_coord, y_coord)

        # Clear the screen (optional)
        screen.fill((255, 255, 255))

        # Draw any additional objects or visuals (optional)
        # For example, you could draw a representation of the joystick position

        # Update the display
        pygame.display.update()

        # A small delay to avoid flooding the serial connection
        time.sleep(0.1)

    pygame.quit()
